# Text to SQL via prompt engineering

Using AI to write SQL quries for us!
(for data creation: http://mockaroo.com)

<br><br>
____
<br>
Retrieving data:

In [1]:
! curl "https://api.mockaroo.com/api/9b72d5e0?count=500&key=925ab950" > "Employees.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35186    0 35186    0     0  24019      0 --:--:--  0:00:01 --:--:-- 24017


Setitng up the db

In [2]:
import sqlite3
import pandas as pd
import os

In [3]:
#SQL schemas
employees_schema = """
CREATE TABLE IF NOT EXISTS Employees (
employee_id INTEGER PRIMARY KEY,
first_name TEXT,
last_name TEXT,
salary INTEGER,
hire_date TEXT,
department_id INTEGER,
department TEXT,
residence_city TEXT,
age INTEGER,
job_level TEXT
);
"""


Putting all our data from the CSV into a db

In [4]:
# Connect to SQLite database
conn = sqlite3.connect('employees.db')
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute(employees_schema)

# Load data from CSV into pandas DataFrame
employees_df = pd.read_csv('Employees.csv')

# Write the data from the DataFrame to the SQLite table
employees_df.to_sql('Employees', conn, if_exists='replace', index=False)

# Verify data insertion
cursor.execute("SELECT COUNT(*) FROM Employees;")
count = cursor.fetchone()[0]
print(f"Number of rows in Employees table: {count}")

# Close the connection
conn.close()

Number of rows in Employees table: 500


Create an API key from google (https://aistudio.google.com/)




In [5]:
!pip install google-genai

Import required modules

In [6]:
from google import genai
from google.colab import userdata

In [8]:
genai_client = genai.Client(api_key=userdata.get('google_api_key'))

Playing around with prompts:

In [9]:
prompt = """
### ROLE

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### CONTEXT

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's employee database using natural language. The database dialect is always SQLite. Your responses will be executed directly on the database.

The database consists of a single table:

Employees table

CREATE TABLE IF NOT EXISTS Employees (
  employee_id INTEGER PRIMARY KEY,
  first_name VARCHAR(20),
  last_name VARCHAR(20),
  salary INT,
  hire_date DATE,
  department_id INT,
  department VARCHAR(20),
  residence_city VARCHAR(20),
  age INT,
  job_level VARCHAR(20)
);

-----

### TASK

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1. Analyze the User's Query: Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like SUM, COUNT, AVG), and ordering they are asking for.
2. Map to the Schema: Map the entities from the user's query to the appropriate columns in the Employees table. Use only the columns provided in the schema.
3. Construct the SQLite Query: Write a clean and efficient SELECT statement that is syntactically correct for SQLite. Ensure all column names are accurate.
4. Handle Ambiguity: If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### CONSTRAINTS

- Read-Only Operations: You must only generate SELECT queries. Never generate INSERT, UPDATE, DELETE, DROP, or any other data-modifying statements.
- Adhere Strictly to Schema: Only use the Employees table and its defined columns. Do not invent or assume the existence of any other tables or columns.
- No Explanations: Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
- Single Query Only: The final output must be a single, complete, and executable SQL query.
- Handle Impossibility: If a request is impossible to fulfill with the given schema (for example, "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### EXAMPLES

Example 1: Simple Lookup
User Query: "Show me all employees who live in Noida"
Expected Output:
{
  "status": "success",
  "response": "SELECT * FROM Employees WHERE residence_city = 'Noida';"
}

Example 2: Aggregation and Sorting
User Query: "What are the top 3 highest-paid employees?"
Expected Output:
{
  "status": "success",
  "response": "SELECT first_name, last_name, salary FROM Employees ORDER BY salary DESC LIMIT 3;"
}

Example 3: Ambiguous Query
User Query: "Show me recent employees"
Expected Output:
{
  "status": "clarification_needed",
  "response": "Could you please define what 'recent' means? For example, 'hired in the last 30 days', 'this year', or 'since August 2025'."
}

Example 4: Impossible Query
User Query: "Which employee made the most sales?"
Expected Output:
{
  "status": "error",
  "response": "I cannot answer this question as the database does not contain information about sales."
}

-----

### OUTPUT FORMAT

Your final response must be a single JSON object with two keys:

1. "status": A string with one of three possible values: "success", "clarification_needed", or "error".
2. "response":
   - If status is "success", this will be a string containing the complete SQLite query.
   - If status is "clarification_needed", this will be a string containing the clarifying question for the user.
   - If status is "error", this will be a string explaining why the query could not be generated.
"""

In [10]:
import json
def get_sql_query(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output


In [11]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='employees.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [12]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [13]:
text2sql(genai_client, prompt, "Show me the number of employees in each department")

Input Token Count: 1001
Thoughts Token Count: 56
Output Token Count: 43
Total Token Count: 1100

Executing query on 'employees.db':
SELECT department, COUNT(employee_id) AS number_of_employees FROM Employees GROUP BY department;
Query executed successfully.


,department,number_of_employees
0,Finance,101
1,HR,95
2,IT,98
3,Marketing,98
4,Sales,108


In [14]:
text2sql(genai_client, prompt, "Show me the employees count by their residence city")

Input Token Count: 1001
Thoughts Token Count: 62
Output Token Count: 51
Total Token Count: 1114

Executing query on 'employees.db':
SELECT residence_city, COUNT(employee_id) AS employee_count FROM Employees GROUP BY residence_city ORDER BY employee_count DESC;
Query executed successfully.


,residence_city,employee_count
0,Zhuxi,2
1,Xuebu,2
2,San Francisco,2
3,Phoenix,2
4,Longtou,2
...,...,...
486,Akora,1
487,Ajaccio,1
488,Aguitu,1
489,Ad Dīwānīyah,1


In [15]:
text2sql(genai_client, prompt, "Who is the highest paid employee and what is their job?")

Input Token Count: 1004
Thoughts Token Count: 81
Output Token Count: 46
Total Token Count: 1131

Executing query on 'employees.db':
SELECT first_name, last_name, salary, job_level FROM Employees ORDER BY salary DESC LIMIT 1;
Query executed successfully.


,first_name,last_name,salary,job_level
0,Concordia,Tugman,149564.12,Mid Level


In [16]:
text2sql(genai_client, prompt, "What is the average age of employees in each department?")

Input Token Count: 1003
Thoughts Token Count: 59
Output Token Count: 39
Total Token Count: 1101

Executing query on 'employees.db':
SELECT department, AVG(age) AS average_age FROM Employees GROUP BY department;
Query executed successfully.


,department,average_age
0,Finance,38.168317
1,HR,38.873684
2,IT,41.877551
3,Marketing,38.785714
4,Sales,38.851852


In [18]:
text2sql(genai_client, prompt, "which departments have the highest and lowest average salary?")

Input Token Count: 1002
Thoughts Token Count: 604
Output Token Count: 75
Total Token Count: 1681

Executing query on 'employees.db':
SELECT department, AVG(salary) AS average_salary FROM Employees GROUP BY department ORDER BY average_salary DESC LIMIT 1 UNION ALL SELECT department, AVG(salary) AS average_salary FROM Employees GROUP BY department ORDER BY average_salary ASC LIMIT 1;
Database error executing query: ORDER BY clause should come after UNION ALL not before


In [21]:
text2sql(genai_client, prompt, "which department has the highest number of employees and how many employees are there in that dept?")

Input Token Count: 1010
Thoughts Token Count: 98
Output Token Count: 50
Total Token Count: 1158

Executing query on 'employees.db':
SELECT department, COUNT(employee_id) AS employee_count FROM Employees GROUP BY department ORDER BY employee_count DESC LIMIT 1;
Query executed successfully.


,department,employee_count
0,Sales,108


In [22]:
text2sql(genai_client, prompt, "How many employees are there in the company?")

Input Token Count: 1001
Thoughts Token Count: 50
Output Token Count: 32
Total Token Count: 1083

Executing query on 'employees.db':
SELECT COUNT(employee_id) FROM Employees;
Query executed successfully.


,COUNT(employee_id)
0,500


In [23]:
text2sql(genai_client, prompt, "Who is the oldest employee in the company? also who is the youngest")

Input Token Count: 1006
Thoughts Token Count: 236
Output Token Count: 63
Total Token Count: 1305

Executing query on 'employees.db':
SELECT first_name, last_name, age FROM Employees ORDER BY age DESC LIMIT 1 UNION ALL SELECT first_name, last_name, age FROM Employees ORDER BY age ASC LIMIT 1;
Database error executing query: ORDER BY clause should come after UNION ALL not before


Hmm...a little prompting shoud do the trick here...

In [24]:
prompt = """
### ROLE

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### CONTEXT

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's employee database using natural language. The database dialect is always SQLite. Your responses will be executed directly on the database.

The database consists of a single table:

Employees table

CREATE TABLE IF NOT EXISTS Employees (
  employee_id INTEGER PRIMARY KEY,
  first_name VARCHAR(20),
  last_name VARCHAR(20),
  salary INT,
  hire_date DATE,
  department_id INT,
  department VARCHAR(20),
  residence_city VARCHAR(20),
  age INT,
  job_level VARCHAR(20)
);

-----

### TASK

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1. Analyze the User's Query: Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like SUM, COUNT, AVG, MAX, MIN), and ordering they are asking for.
2. Map to the Schema: Map the entities from the user's query to the appropriate columns in the Employees table. Use only the columns provided in the schema.
3. Construct the SQLite Query: Write a clean and efficient SELECT statement that is syntactically correct for SQLite. Ensure all column names are accurate.
   - **Important:** If combining multiple SELECT statements (like oldest and youngest employees), either use MAX()/MIN() or wrap each SELECT in parentheses before applying UNION or UNION ALL. Avoid using ORDER BY before UNION.
4. Handle Ambiguity: If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### CONSTRAINTS

- Read-Only Operations: You must only generate SELECT queries. Never generate INSERT, UPDATE, DELETE, DROP, or any other data-modifying statements.
- Adhere Strictly to Schema: Only use the Employees table and its defined columns. Do not invent or assume the existence of any other tables or columns.
- No Explanations: Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
- Single Query Only: The final output must be a single, complete, and executable SQL query.
- Handle Impossibility: If a request is impossible to fulfill with the given schema (for example, "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### EXAMPLES

Example 1: Simple Lookup
User Query: "Show me all employees who live in Noida"
Expected Output:
{
  "status": "success",
  "response": "SELECT * FROM Employees WHERE residence_city = 'Noida';"
}

Example 2: Aggregation and Sorting
User Query: "What are the top 3 highest-paid employees?"
Expected Output:
{
  "status": "success",
  "response": "SELECT first_name, last_name, salary FROM Employees ORDER BY salary DESC LIMIT 3;"
}

Example 3: Oldest and Youngest Employees
User Query: "Who is the oldest employee in the company? Also who is the youngest?"
Expected Output:
{
  "status": "success",
  "response": "SELECT first_name, last_name, age FROM Employees WHERE age = (SELECT MAX(age) FROM Employees) UNION ALL SELECT first_name, last_name, age FROM Employees WHERE age = (SELECT MIN(age) FROM Employees);"
}

Example 4: Ambiguous Query
User Query: "Show me recent employees"
Expected Output:
{
  "status": "clarification_needed",
  "response": "Could you please define what 'recent' means? For example, 'hired in the last 30 days', 'this year', or 'since August 2025'."
}

Example 5: Impossible Query
User Query: "Which employee made the most sales?"
Expected Output:
{
  "status": "error",
  "response": "I cannot answer this question as the database does not contain information about sales."
}

-----

### OUTPUT FORMAT

Your final response must be a single JSON object with two keys:

1. "status": A string with one of three possible values: "success", "clarification_needed", or "error".
2. "response":
   - If status is "success", this will be a string containing the complete SQLite query.
   - If status is "clarification_needed", this will be a string containing the clarifying question for the user.
   - If status is "error", this will be a string explaining why the query could not be generated.
"""


In [25]:
text2sql(genai_client, prompt, "Who is the oldest employee in the company? also who is the youngest")

Input Token Count: 1158
Thoughts Token Count: 195
Output Token Count: 72
Total Token Count: 1425

Executing query on 'employees.db':
SELECT first_name, last_name, age FROM Employees WHERE age = (SELECT MAX(age) FROM Employees) UNION ALL SELECT first_name, last_name, age FROM Employees WHERE age = (SELECT MIN(age) FROM Employees);
Query executed successfully.


,first_name,last_name,age
0,Thom,Adair,60
1,Luke,Litherland,60
2,Farlee,Kerr,60
3,Mychal,Rospars,60
4,Kalle,Norledge,60
5,Olivia,Roelofsen,60
6,Bernardina,Cobbing,18
7,Trueman,Ingold,18
8,Arliene,Castanos,18
9,Steward,Mixer,18


There we go!! :D

In [29]:
text2sql(genai_client, prompt, "heelo - which is the best employee?")

Input Token Count: 1153
Thoughts Token Count: 168
Output Token Count: 60
Total Token Count: 1381


{'status': 'error',
 'response': "I cannot answer this question as the database does not contain information to determine the 'best' employee (e.g., performance metrics, sales figures, or specific criteria for 'best')."}